In [ ]:
import numpy as np
import pickle
import healpy as hp
import matplotlib.pyplot as plt
import sys
import os
import subprocess
sys.path.append('../shared')
sys.path.append('../analytic_model')
from input import Info
from generate_maps import generate_freq_maps
from load_weight_maps import load_wt_maps
from utils import setup_output_dir, tsz_spectral_response, GaussianNeedlets, build_NILC_maps

In [ ]:
font = {'size'   : 20, 'family':'STIXGeneral'}
plt.rcParams.update({
     'text.usetex': True,
     'font.family': 'serif',
     'font.sans-serif': ['Computer Modern'],
     'font.size':20})
plt.rc_context({'axes.autolimit_mode': 'round_numbers'})

### To run this notebook, first run analytic/main.py with save_files=1 in the yaml file. The variable save_dir below should be the directory in which to save the plots (set save_dir=None if you don't want to save the plots).

In [ ]:
save_dir = 'analytic_plots'

sim = 0
if not os.path.isdir(save_dir):
    env = os.environ.copy()
    subprocess.call(f'mkdir {save_dir}', shell=True, env=env)



## Read input yaml file

In [ ]:
# main input file containing most specifications 
input_file = '/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/analytic_model/laptop.yaml'

# read in the input file and set up relevant info object
inp = Info(input_file)

## Sample input maps and power spectra

In [ ]:
cmb_cl, tsz_cl, noise1_cl, noise2_cl, cmb_map, tsz_map, noise1_map, noise2_map = generate_freq_maps(sim, inp, save=False)
ells_sum = np.arange(inp.ell_sum_max+1)

g_cmb = np.array([1., 1.])
g_tsz = tsz_spectral_response(inp.freqs)

fig, axs = plt.subplots(2,2, figsize=(12,8))
axs = axs.flatten()
margins=None #(1,1,1,1)

plt.rcParams.update({
     'text.usetex': True,
     'font.family': 'serif',
     'font.sans-serif': ['Computer Modern'],
     'font.size':14})

for n, ax in enumerate(axs):
    plt.axes(ax)
    if n==0:
        hp.mollview(cmb_map, fig=1, hold=True, title='CMB', format='%.03g')
    elif n==1:
        hp.mollview(tsz_map, fig=2, hold=True, title=r'Amplified Compton-$y$', format='%.03g')
    elif n==2:
        hp.mollview(noise1_map, fig=3, hold=True, title='Noise 90 GHz', format='%.03g')
    else:
        to_dl = ells_sum*(ells_sum+1)/2/np.pi
        plt.plot(ells_sum[2:], (to_dl*cmb_cl)[2:], label='CMB')
        plt.plot(ells_sum[2:], abs(to_dl*g_tsz[0]*tsz_cl)[2:], label=r'$|$Amplified tSZ 90 GHz$|$', linestyle='solid')
        plt.plot(ells_sum[2:], abs(to_dl*g_tsz[1]*tsz_cl)[2:], label=r'$|$Amplified tSZ 150 GHz$|$', linestyle='solid')
        plt.plot(ells_sum[2:], (to_dl*noise1_cl)[2:], label='Noise 90 GHz', linestyle='solid')
        plt.plot(ells_sum[2:], (to_dl*noise2_cl)[2:], label='Noise 150 GHz', linestyle='solid')
        plt.xlabel(r'$\ell$', fontsize=14)
        plt.ylabel(r'$\ell(\ell+1)C_{\ell}/(2\pi)$ [$\mathrm{K}^2$]', fontsize=14)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.grid()
        plt.legend(fontsize=12, ncol=2, loc=(0.01,0.01))
        plt.yscale('log')
        plt.xlim(2,inp.ell_sum_max+1)
    if n==0 or n==1 or n==2:
        plt.text(0.47, -0.02, r'$[\mathrm{K}]$', transform=ax.transAxes,size=14)

if save_dir:
    plt.savefig(f'{save_dir}/input_maps.pdf', bbox_inches="tight")
    print(f'saved {save_dir}/input_maps.pdf')


plt.rcParams.update({
     'text.usetex': True,
     'font.family': 'serif',
     'font.sans-serif': ['Computer Modern'],
     'font.size':20})

## Needlet filters

In [ ]:
ells_sum, h = GaussianNeedlets(inp)
for n in range(inp.Nscales):
    plt.plot(ells_sum, h[n], color='k')
plt.grid()
plt.xlim(0,inp.ell_sum_max)
plt.xlabel(r'$\ell$')
plt.ylabel(r'$h_\ell^{(n)}$')
if save_dir:
    plt.savefig(f'{save_dir}/needlet_filters.pdf')
    print(f'saved {save_dir}/needlet_filters.pdf')


## Weight maps for preserved CMB component

In [ ]:
CMB_wt_maps, tSZ_wt_maps = load_wt_maps(inp, sim)

In [ ]:
plt.rcParams.update({'font.size':11})

fig, axs = plt.subplots(inp.Nscales, len(inp.freqs), figsize=(8,6))
axs = axs.flatten()

ax = 0
for n in range(inp.Nscales):
    for i in range(len(inp.freqs)):
        plt.axes(axs[ax])
        hp.mollview(CMB_wt_maps[n][i], hold=True, title='', format='%.03g')
        if i == 0:
            plt.text(0.85, 0.5, f'Scale {n}\n{int(inp.freqs[i])} GHz', transform=axs[ax].transAxes,size=11)
        elif i == 1:
            plt.text(-0.04, 0.5, f'Scale {n}\n{int(inp.freqs[i])} GHz', transform=axs[ax].transAxes,size=11)
        ax += 1

if save_dir:
    plt.savefig(f'{save_dir}/weight_maps.pdf')
    print(f'saved {save_dir}/weight_maps.pdf')
    
plt.rcParams.update({'font.size':20})

## Component propagation to NILC map (comparing analytic and directly calculated results)

In [ ]:
plt.rcParams.update({'font.size':14})

#index as directly_computed_prop_to_NILC_PS[z,p,l]
directly_computed_prop_to_NILC_PS = pickle.load(open(f'{inp.output_dir}/data_vecs/sim{sim}_directly_computed_prop_to_NILC_PS.p', 'rb'))

#index as Clpq[p,q,z,reMASTERed term,l]
Clpq = pickle.load(open(f'{inp.output_dir}/data_vecs/sim{sim}_Clpq.p', 'rb'))

#index as 0-2 for CMB, tSZ, noise
Clzz = pickle.load(open(f'{inp.output_dir}/n_point_funcs/sim{sim}_Clzz.p', 'rb'))

comps = ['CMB', 'ftSZ']

ells = np.arange(inp.ellmax+1)
start = 2

fig, axs = plt.subplots(2,2, figsize=(9,6))
axs = axs.flatten()
ax = 0
for z, prop_comp in enumerate(comps):
    for p, preserved_comp in enumerate(comps):
        plt.axes(axs[ax])
        direct = directly_computed_prop_to_NILC_PS[z,p]
        remastered_terms = Clpq[p,p,z]
        analytic = np.sum(remastered_terms, axis=0)
        aa_ww_term, aw_aw_term, w_aaw_term, a_waw_term, aaww_term = remastered_terms
        plt.plot(ells[start:], (ells*(ells+1)*aa_ww_term/(2*np.pi))[start:], label=r'$\langle zz \rangle \langle ww \rangle$ term', color='c')
        plt.plot(ells[start:], (ells*(ells+1)*aw_aw_term/(2*np.pi))[start:], label=r'$\langle zw \rangle \langle zw \rangle$ term')
        plt.plot(ells[start:], (ells*(ells+1)*w_aaw_term/(2*np.pi))[start:], label=r'$\langle w \rangle \langle zzw \rangle_c$ terms', color='r')
        plt.plot(ells[start:], (ells*(ells+1)*a_waw_term/(2*np.pi))[start:], label=r'$\langle z \rangle \langle wzw \rangle_c$ terms', color='mediumpurple', linestyle='dashed')
        plt.plot(ells[start:], (ells*(ells+1)*aaww_term/(2*np.pi))[start:], label=r'$\langle zzww \rangle_c$ term', color='y', linestyle='dotted')
        plt.plot(ells[start:], (ells*(ells+1)*direct/(2*np.pi))[start:], label='Directly Computed', color='g')
        if ax==0:
            plt.plot(ells[start:], (ells*(ells+1)*cmb_cl/(2*np.pi))[start:], label='Input Spectrum', linestyle='dashed', color='lightblue')
        elif ax==3:
            plt.plot(ells[start:], (ells*(ells+1)*tsz_cl/(2*np.pi))[start:], label='Input Spectrum', linestyle='dashed', color='lightblue')
        plt.plot(ells[start:], (ells*(ells+1)*analytic/(2*np.pi))[start:], label='Analytic Result', linestyle='dotted', color='m')
        plt.xlabel(r'$\ell$')
        plt.ylabel(r'$D_\ell$ $[\mathrm{K}^2]$')
        plt.grid()
        plt.xlim(2,inp.ellmax)
        plt.title(f'{comps[z]} Propagation to {comps[p]} NILC PS', fontsize=14)
        ax += 1
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, fontsize=12, bbox_to_anchor=(0.9, 0.04), ncol=3)
plt.tight_layout()
fig = plt.gcf()
fig.subplots_adjust(top=0.85)

if save_dir:
    plt.savefig(f'{save_dir}/propagation.pdf', bbox_inches='tight')
    print(f'saved {save_dir}/propagation.pdf')

plt.rcParams.update({'font.size':20})

